In [2]:
apikey = 'NR59H1H1GHPHO7ZP'
from requests import Request, Session
import pandas as pd
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import datetime

In [1]:
import json
import time
import datetime
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

def get_historical_news():
    apikey = 'NR59H1H1GHPHO7ZP'
    path = "C:/Users/Admin/OneDrive - VNU-HCMUS/Documents/K1N4/E2E/ETL/data/raw/news/historical/news_data_historical.json"
    session = Session()

    final_news = []

    # Mốc dừng
    time_from_limit = datetime.datetime.strptime("20240505T0130", "%Y%m%dT%H%M")

    next_time_to = None  # chưa có time_to ở lần đầu
    base_time_from = time_from_limit.strftime("%Y%m%dT%H%M")

    while True:
        if next_time_to:
            time_to_str = next_time_to.strftime("%Y%m%dT%H%M")
            print(f"Fetching news from {base_time_from} to {time_to_str}")
            url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=COIN,CRYPTO:BTC,FOREX:USD&time_from={base_time_from}&time_to={time_to_str}&limit=1000&apikey={apikey}'
        else:
            print(f"Fetching news from {base_time_from} to now")
            url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=COIN,CRYPTO:BTC,FOREX:USD&time_from={base_time_from}&limit=1000&apikey={apikey}'

        try:
            response = session.get(url)
            response.raise_for_status()
            data = response.json()

            if 'feed' not in data or not data['feed']:
                print("No more news found.")
                print(data)
                break

            articles = data['feed']
            final_news.extend(articles)

            oldest_article_time = min(
                datetime.datetime.strptime(article['time_published'], "%Y%m%dT%H%M%S")
                for article in articles
            )

            print(f"Fetched {len(articles)} articles, oldest at {oldest_article_time}")

            if oldest_article_time <= time_from_limit:
                print("Reached time_from limit.")
                break

            # Giảm 1 phút cho vòng sau
            next_time_to = oldest_article_time - datetime.timedelta(minutes=1)

            time.sleep(1)
        except (ConnectionError, Timeout, TooManyRedirects) as e:
            print(f"Request error: {e}")
            break
        except Exception as ex:
            print(f"Unexpected error: {ex}")
            break

    with open(path, 'w') as f:
        json.dump({'feeds': final_news}, f, indent=4)

    print(f"✅ Saved {len(final_news)} articles to {path}")

get_historical_news()



Fetching news from 20240505T0130 to now
Fetched 40 articles, oldest at 2025-03-31 00:41:08
Fetching news from 20240505T0130 to 20250331T0040
Fetched 13 articles, oldest at 2025-01-22 04:07:00
Fetching news from 20240505T0130 to 20250122T0406
Fetched 24 articles, oldest at 2024-10-23 07:51:21
Fetching news from 20240505T0130 to 20241023T0750
Fetched 17 articles, oldest at 2024-07-23 18:45:00
Fetching news from 20240505T0130 to 20240723T1844
Fetched 36 articles, oldest at 2024-05-15 12:13:08
Fetching news from 20240505T0130 to 20240515T1212
Fetched 2 articles, oldest at 2024-05-06 20:32:14
Fetching news from 20240505T0130 to 20240506T2031
No more news found.
{'items': '0', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': []}
✅ Saved 132 articles to C:/Users/Admin/O